In [35]:
# import statements
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [36]:
# assessing the .tsv file that holds the kinematic data ({x,y,z} coordinates for each marker)
file_3D = pd.read_csv('Trial0001_static.tsv', delimiter='\t', skiprows=10)

file_3D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Frame             129 non-null    int64  
 1   Time              129 non-null    float64
 2   1 - Head X        129 non-null    float64
 3   1 - Head Y        129 non-null    float64
 4   1 - Head Z        129 non-null    float64
 5   2 - L_Shoulder X  129 non-null    float64
 6   2 - L_Shoulder Y  129 non-null    float64
 7   2 - L_Shoulder Z  129 non-null    float64
 8   3 - L_Elbow X     129 non-null    float64
 9   3 - L_Elbow Y     129 non-null    float64
 10  3 - L_Elbow Z     129 non-null    float64
 11  4 - L_Wrist X     129 non-null    float64
 12  4 - L_Wrist Y     129 non-null    float64
 13  4 - L_Wrist Z     129 non-null    float64
 14  5 - R_Shoulder X  129 non-null    float64
 15  5 - R_Shoulder Y  129 non-null    float64
 16  5 - R_Shoulder Z  129 non-null    float64
 1

In [37]:
file_3D.head()

,Frame,Time,1 - Head X,1 - Head Y,1 - Head Z,2 - L_Shoulder X,2 - L_Shoulder Y,2 - L_Shoulder Z,3 - L_Elbow X,3 - L_Elbow Y,...,17 - R_Heel X,17 - R_Heel Y,17 - R_Heel Z,18 - R_Meta_V X,18 - R_Meta_V Y,18 - R_Meta_V Z,19 - R_Toe_II X,19 - R_Toe_II Y,19 - R_Toe_II Z,Unnamed: 59
0,119,1.18,341.204,299.258,1814.116,317.832,510.946,1488.631,301.931,562.535,...,236.033,251.099,22.720,399.810,167.430,23.294,499.254,204.631,20.032,NaN
1,120,1.19,341.176,299.248,1814.007,317.728,510.856,1488.634,301.864,562.602,...,236.033,251.101,22.707,399.861,167.511,23.330,499.280,204.685,20.000,NaN
2,121,1.20,341.355,299.344,1814.062,317.745,510.905,1488.553,301.766,562.595,...,235.992,251.097,22.660,399.755,167.580,23.196,499.174,204.691,19.999,NaN
3,122,1.21,341.324,299.194,1814.076,317.803,510.987,1488.576,301.684,562.639,...,235.944,251.112,22.760,399.724,167.554,23.251,499.276,204.633,19.994,NaN
4,123,1.22,341.528,299.236,1814.164,317.754,510.966,1488.644,301.454,562.650,...,235.973,251.127,22.785,399.857,167.493,23.250,499.313,204.650,19.970,NaN


In [38]:
# Project 3D data onto 2D plane (dropping y-axis information)
columns_to_drop = [col for col in file_3D.columns if ' Y' in col]

# Drop the columns from the DataFrame
# The `axis=1` tells pandas to drop columns, not rows.
static_2D = file_3D.drop(columns=columns_to_drop, axis=1)

selected_columns = static_2D.columns[2:]

for col in selected_columns:
    static_2D[col] = static_2D[col]/1000

print(static_2D.head())

   Frame  Time  1 - Head X  1 - Head Z  2 - L_Shoulder X  2 - L_Shoulder Z  \
0    119  1.18    0.341204    1.814116          0.317832          1.488631   
1    120  1.19    0.341176    1.814007          0.317728          1.488634   
2    121  1.20    0.341355    1.814062          0.317745          1.488553   
3    122  1.21    0.341324    1.814076          0.317803          1.488576   
4    123  1.22    0.341528    1.814164          0.317754          1.488644   

   3 - L_Elbow X  3 - L_Elbow Z  4 - L_Wrist X  4 - L_Wrist Z  ...  \
0       0.301931       1.127307       0.436824       0.908965  ...   
1       0.301864       1.127290       0.436687       0.908941  ...   
2       0.301766       1.127298       0.436535       0.908999  ...   
3       0.301684       1.127384       0.436514       0.908884  ...   
4       0.301454       1.127356       0.436458       0.908880  ...   

   15 - R_Knee Z  16 - R_Ankle X  16 - R_Ankle Z  17 - R_Heel X  \
0       0.512978        0.286463        0.0

In [39]:
def create_coordinate_columns(df):
    """
    Transforms separate 'Marker X' and 'Marker Z' columns into
    a single 'Marker_Coord' column of (X, Z) tuples.
    """
    # Get all column names as a list
    all_columns = df.columns.tolist()

    # Identify unique marker bases (e.g., '1 - Head', '2 - L_Shoulder')
    marker_bases = set()
    for col in all_columns:
        if col.endswith(' X') or col.endswith(' Z'):
            # The base is everything up to the last space,
            # e.g., '1 - Head' from '1 - Head X'
            base = col[:-2].strip()
            marker_bases.add(base)

    # Create the new DataFrame for coordinates
    coord_df = pd.DataFrame()

    # Iterate through each identified marker base and create the coordinate column
    for base in sorted(list(marker_bases)):
        x_col = f'{base} X'
        z_col = f'{base} Z'
        coord_col = f'{base}_Coord'

        # Ensure both X and Z columns exist for this base
        if x_col in df.columns and z_col in df.columns:
            coord_df[coord_col] = df.apply(
                lambda row: (row[x_col], row[z_col]),
                axis=1
            )

    return coord_df

# 3. Create the new DataFrame with coordinate tuples
coord_df = create_coordinate_columns(static_2D)

print("\nDataFrame with only Coordinate Tuples:\n", coord_df)


DataFrame with only Coordinate Tuples:
                       1 - Head_Coord  \
0               (0.341204, 1.814116)   
1     (0.341176, 1.8140070000000001)   
2     (0.341355, 1.8140619999999998)   
3               (0.341324, 1.814076)   
4     (0.341528, 1.8141639999999999)   
..                               ...   
124              (0.35137, 1.813242)   
125             (0.351522, 1.813235)   
126             (0.351537, 1.813234)   
127  (0.35159500000000005, 1.813254)   
128             (0.351764, 1.813226)   

                             10 - L_Ankle_Coord  \
0               (0.28139400000000003, 0.064485)   
1                          (0.281313, 0.064558)   
2                          (0.281311, 0.064496)   
3                          (0.281435, 0.064399)   
4               (0.281423, 0.06448000000000001)   
..                                          ...   
124  (0.28136700000000003, 0.06446299999999999)   
125                          (0.281289, 0.0644)   
126             (0.

In [40]:
def calculate_segment_com(df, proximal_col, distal_col, com_ratio):
    """
    Calculates the CoM coordinates (x, z) for a segment.
    Returns: A two series of x_com, z_com
    """
    # 1. Extract X and Z components
    x1 = df[proximal_col].apply(lambda t: t[0])
    z1 = df[proximal_col].apply(lambda t: t[1])
    x2 = df[distal_col].apply(lambda t: t[0])
    z2 = df[distal_col].apply(lambda t: t[1])

    # 2. Calculate CoM (Position) - Returns two Series
    x_com = x1 + (x2 - x1) * com_ratio
    z_com = z1 + (z2 - z1) * com_ratio

    return x_com, z_com

In [41]:
def package_coordinates(x_series, z_series):
    """
    Takes two pandas Series (x and z) and packages them into a single
    Series of (x, z) tuples.
    """
    return pd.Series(list(zip(x_series, z_series)))

In [42]:
# Composing Dataframe 'Points' for biomechanical model
num_rows_static = len(static_2D)

# Define the column names
column_names_points = [f'P{i}' for i in range(1, 16)] # Generates ['P1', 'P2', ..., 'P14']

# 2. Create an array of NaNs with the correct shape (Rows x Columns)
# The data type is set to float (dtype='float64')
data = np.full((num_rows_static, len(column_names_points)), np.nan)

# 3. Create the final DataFrame
static_points = pd.DataFrame(
    data,
    columns=column_names_points
)

# --- Display the result ---
print("\n--- New Target DataFrame Layout ---")
print(f"Shape: {static_points.shape} (Rows x Columns)")
print(static_points.head()) # Shows the first few rows

# Determining and storing shoulder midpoint data to points dataframe
x_com_shoulder, z_com_shoulder = calculate_segment_com(
    coord_df,
    proximal_col='2 - L_Shoulder_Coord',
    distal_col='5 - R_Shoulder_Coord',
    com_ratio=0.5
)
static_points['P2'] = package_coordinates(x_com_shoulder, z_com_shoulder)

x_com_hip, z_com_hip = calculate_segment_com(
    coord_df,
    proximal_col='8 - L_Hip_Coord',
    distal_col='14 - R_Hip_Coord',
    com_ratio=0.5
)
static_points['P7'] = package_coordinates(x_com_hip, z_com_hip)

print(static_points.head())


--- New Target DataFrame Layout ---
Shape: (129, 15) (Rows x Columns)
   P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  P11  P12  P13  P14  P15
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
   P1                                P2  P3  P4  P5  P6  \
0 NaN   (0.3229205, 1.4893230000000002) NaN NaN NaN NaN   
1 NaN   (0.3228465, 1.4892975000000002) NaN NaN NaN NaN   
2 NaN             (0.3228525, 1.489244) NaN NaN NaN NaN   
3 NaN   (0.32285600000000003, 1.489235) NaN NaN NaN NaN   
4 NaN  (0.32285050000000004, 1.4892805) NaN NaN NaN NaN   

                                 P7  P8  P9  P10  P11  P12  P13  P14  P15  
0              (0.327061, 0.932978) NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  (0.32708499

In [43]:
static_points['P1'] = coord_df['1 - Head_Coord']
static_points['P3'] = coord_df['3 - L_Elbow_Coord']
static_points['P4'] = coord_df['4 - L_Wrist_Coord']
static_points['P5'] = coord_df['6 - R_Elbow_Coord']
static_points['P6'] = coord_df['7 - R_Wrist_Coord']
static_points['P8'] = coord_df['15 - R_Knee_Coord']
static_points['P9'] = coord_df['16 - R_Ankle_Coord']
static_points['P10'] = coord_df['18 - R_Meta_V_Coord']
static_points['P11'] = coord_df['19 - R_Toe_II_Coord']
static_points['P12'] = coord_df['9 - L_Knee_Coord']
static_points['P13'] = coord_df['10 - L_Ankle_Coord']
static_points['P14'] = coord_df['12 - L_Meta_V_Coord']
static_points['P15'] = coord_df['13 - L_Toe_II_Coord']

print(static_points)

                                  P1                                P2  \
0               (0.341204, 1.814116)   (0.3229205, 1.4893230000000002)   
1     (0.341176, 1.8140070000000001)   (0.3228465, 1.4892975000000002)   
2     (0.341355, 1.8140619999999998)             (0.3228525, 1.489244)   
3               (0.341324, 1.814076)   (0.32285600000000003, 1.489235)   
4     (0.341528, 1.8141639999999999)  (0.32285050000000004, 1.4892805)   
..                               ...                               ...   
124              (0.35137, 1.813242)             (0.328691, 1.4893805)   
125             (0.351522, 1.813235)             (0.3287815, 1.489318)   
126             (0.351537, 1.813234)            (0.3287525, 1.4893065)   
127  (0.35159500000000005, 1.813254)             (0.328847, 1.4893585)   
128             (0.351764, 1.813226)                (0.328786, 1.4894)   

                                  P3                               P4  \
0               (0.301931, 1.127307)  

## Define Filter

In [44]:
from scipy import signal
# --- Filter Specifications ---
fc = 6.0    # Cutoff frequency in Hz
fs = 100.0  # Sample frequency in Hz
order = 2   # Second-order Butterworth filter (effective order will be 4 for filtfilt)

# 1. Design the filter and get coefficients
nyquist = fs / 2.0
Wn = fc / nyquist
# 'b' are the numerator coefficients, 'a' are the denominator coefficients
b, a = signal.butter(order, Wn, btype='low', analog=False)

In [45]:
dummy_df = pd.DataFrame()

static_filtered = pd.DataFrame(columns=column_names_points)

for column in column_names_points:

    dummy_df[['X', 'Z']] = static_points[column].apply(pd.Series)

    x_filtered = signal.filtfilt(b, a, dummy_df['X'])
    z_filtered = signal.filtfilt(b, a, dummy_df['Z'])

    static_filtered[column] = package_coordinates(x_filtered, z_filtered)


print(static_filtered)

                                            P1  \
0    (0.34119488529525255, 1.8141174243045737)   
1     (0.34125946462091944, 1.814116595117878)   
2     (0.34132431279936615, 1.814116959931554)   
3    (0.34138912603521204, 1.8141188407547693)   
4     (0.3414531684581295, 1.8141213700780023)   
..                                         ...   
124   (0.3514489869100353, 1.8132447252797568)   
125   (0.35152874169647047, 1.813239768403028)   
126  (0.35161166978629216, 1.8132348515113523)   
127  (0.35169724062098984, 1.8132298837249858)   
128    (0.3517836225469356, 1.813224867770511)   

                                            P2  \
0      (0.3229212172185097, 1.489322090302981)   
1    (0.32290269144379924, 1.4893076934522052)   
2      (0.3228860044848382, 1.489295057221951)   
3    (0.32287238238370786, 1.4892855294929188)   
4     (0.32286238389103905, 1.489279733073011)   
..                                         ...   
124    (0.3287113010759713, 1.489363763014708)   

In [46]:
def calculate_average_distance_from_coords(coord_df, marker1_coord_col, marker2_coord_col):
    """
    Calculates the Euclidean distance between two marker coordinate columns.

    Args:
        coord_df (pd.DataFrame): The DataFrame containing columns of (X, Z) tuples.
        marker1_coord_col (str): The full name of the first coordinate column (e.g., '1 - Head_Coord').
        marker2_coord_col (str): The full name of the second coordinate column (e.g., '2 - L_Shoulder_Coord').

    Returns:
        float: The average Euclidean distance between the two markers.
    """

    # 1. Use the NumPy vectorized approach again
    # This is done by extracting the X (index 0) and Z (index 1) components
    # from the tuples in the pandas Series.

    # Vectorized extraction of X and Z components
    x1 = coord_df[marker1_coord_col].str[0]
    z1 = coord_df[marker1_coord_col].str[1]

    x2 = coord_df[marker2_coord_col].str[0]
    z2 = coord_df[marker2_coord_col].str[1]

    # 2. Calculate the Euclidean Distance (vectorized)
    distances = np.sqrt((x2 - x1)**2 + (z2 - z1)**2)

    # 3. Calculate and return the average
    return distances.mean()

In [47]:
# Right side of body
column_names_avg_length = ['Neck', 'Trunk', 'Right Upper Arm', 'Right Forearm', 'Left Upper Arm', 'Left Forearm', 'Right Thigh', 'Right Shank', 'Right Foot', 'Right Toe', 'Left Thigh', 'Left Shank', 'Left Foot', 'Left Toe']

empty_array  = np.full((3, len(column_names_avg_length)), np.nan)

avg_lengths = pd.DataFrame(
    data=empty_array,
    columns=column_names_avg_length
)

avg_lengths.loc[0, 'Right Upper Arm'] = calculate_average_distance_from_coords(static_filtered, 'P2','P5')
avg_lengths.loc[0, 'Right Forearm'] = calculate_average_distance_from_coords(static_filtered,'P5','P6')

avg_lengths.loc[0, 'Right Thigh'] = calculate_average_distance_from_coords(static_filtered,'P7','P8')
avg_lengths.loc[0, 'Right Shank'] = calculate_average_distance_from_coords(static_filtered,'P8','P9')
avg_lengths.loc[0, 'Right Foot'] = calculate_average_distance_from_coords(static_filtered,'P9','P10')
avg_lengths.loc[0, 'Right Toe'] = calculate_average_distance_from_coords(static_filtered,'P10','P11')

# Left side of body
avg_lengths.loc[0, 'Left Upper Arm'] = calculate_average_distance_from_coords(static_filtered, 'P2','P3')
avg_lengths.loc[0, 'Left Forearm'] = calculate_average_distance_from_coords(static_filtered,'P3','P4')

avg_lengths.loc[0, 'Left Thigh'] = calculate_average_distance_from_coords(static_filtered,'P7','P12')
avg_lengths.loc[0, 'Left Shank'] = calculate_average_distance_from_coords(static_filtered,'P12','P13')
avg_lengths.loc[0, 'Left Foot'] = calculate_average_distance_from_coords(static_filtered,'P13','P14')
avg_lengths.loc[0, 'Left Toe'] = calculate_average_distance_from_coords(static_filtered,'P14','P15')

# Middle part of the body
avg_lengths.loc[0, 'Neck'] = calculate_average_distance_from_coords(static_filtered,'P1','P2')
avg_lengths.loc[0, 'Trunk'] = calculate_average_distance_from_coords(static_filtered,'P2','P7')

print(avg_lengths)

       Neck     Trunk  Right Upper Arm  Right Forearm  Left Upper Arm  \
0  0.325111  0.556323         0.366535       0.248372        0.362744   
1       NaN       NaN              NaN            NaN             NaN   
2       NaN       NaN              NaN            NaN             NaN   

   Left Forearm  Right Thigh  Right Shank  Right Foot  Right Toe  Left Thigh  \
0      0.256846     0.419931      0.44707    0.121699   0.099472    0.420626   
1           NaN          NaN          NaN         NaN        NaN         NaN   
2           NaN          NaN          NaN         NaN        NaN         NaN   

   Left Shank  Left Foot  Left Toe  
0    0.449373   0.121326  0.095845  
1         NaN        NaN       NaN  
2         NaN        NaN       NaN  


In [48]:
def calculate_segment_com_lengths(average_length_abs, com_ratio_relative):
    """
    Splits the absolute segment length based on a relative CoM ratio.

    Parameters:
    - average_length_abs (float): The total absolute length of the segment (e.g., avg trunk length in meters).
    - com_ratio_relative (float): The relative position of the CoM
                                  (e.g., 0.433 means 43.3% from the proximal end).

    Returns:
    - tuple: (length_proximal_to_com, length_com_to_distal)
    """
    if com_ratio_relative < 0 or com_ratio_relative > 1:
        raise ValueError("CoM ratio must be between 0 and 1 (inclusive).")

    # 1. Calculate the length from the PROXIMAL end to the CoM (L_p_to_com)
    # L_p_to_com = Total Length * Ratio
    length_proximal_to_com = average_length_abs * com_ratio_relative

    # 2. Calculate the length from the CoM to the DISTAL end (L_com_to_d)
    # L_com_to_d = Total Length - L_p_to_com
    length_com_to_distal = average_length_abs - length_proximal_to_com

    return length_proximal_to_com, length_com_to_distal

In [49]:
avg_lengths.loc[1, 'Neck'], avg_lengths.loc[2, 'Neck'] = calculate_segment_com_lengths(avg_lengths.loc[0, 'Neck'], 0.5)
avg_lengths.loc[1, 'Trunk'], avg_lengths.loc[2, 'Trunk'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Trunk'], 0.5)

avg_lengths.loc[1, 'Right Upper Arm'], avg_lengths.loc[2, 'Right Upper Arm'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Right Upper Arm'], 0.436)
avg_lengths.loc[1, 'Left Upper Arm'], avg_lengths.loc[2, 'Left Upper Arm'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Left Upper Arm'], 0.436)

avg_lengths.loc[1, 'Right Forearm'], avg_lengths.loc[2, 'Right Forearm'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Right Forearm'], 0.430)
avg_lengths.loc[1, 'Left Forearm'], avg_lengths.loc[2, 'Left Forearm'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Left Forearm'], 0.430)

avg_lengths.loc[1, 'Right Thigh'], avg_lengths.loc[2, 'Right Thigh'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Right Thigh'], 0.433)
avg_lengths.loc[1, 'Left Thigh'], avg_lengths.loc[2, 'Left Thigh'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Left Thigh'], 0.433)

avg_lengths.loc[1, 'Right Shank'], avg_lengths.loc[2, 'Right Shank'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Right Shank'], 0.433)
avg_lengths.loc[1, 'Left Shank'], avg_lengths.loc[2, 'Left Shank'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Left Shank'], 0.433)

avg_lengths.loc[1, 'Right Foot'], avg_lengths.loc[2, 'Right Foot'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Right Foot'], 0.5)
avg_lengths.loc[1, 'Left Foot'], avg_lengths.loc[2, 'Left Foot'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Left Foot'], 0.5)

avg_lengths.loc[1, 'Right Toe'], avg_lengths.loc[2, 'Right Toe'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Right Toe'], 0.5)
avg_lengths.loc[1, 'Left Toe'], avg_lengths.loc[2, 'Left Toe'] = calculate_segment_com_lengths(avg_lengths.loc[0,'Left Toe'], 0.5)
#
print(avg_lengths)


       Neck     Trunk  Right Upper Arm  Right Forearm  Left Upper Arm  \
0  0.325111  0.556323         0.366535       0.248372        0.362744   
1  0.162556  0.278162         0.159809       0.106800        0.158156   
2  0.162556  0.278162         0.206726       0.141572        0.204588   

   Left Forearm  Right Thigh  Right Shank  Right Foot  Right Toe  Left Thigh  \
0      0.256846     0.419931     0.447070    0.121699   0.099472    0.420626   
1      0.110444     0.181830     0.193581    0.060850   0.049736    0.182131   
2      0.146402     0.238101     0.253489    0.060850   0.049736    0.238495   

   Left Shank  Left Foot  Left Toe  
0    0.449373   0.121326  0.095845  
1    0.194579   0.060663  0.047922  
2    0.254795   0.060663  0.047922  


In [50]:
def create_joint_length_array(df, proximal_segment_col, distal_segment_col):
    """
    Creates a 4-element array for the joint lengths based on the CoM sub-lengths.

    Assumptions (based on your DataFrame layout):
    - Row 0: Total Segment Length
    - Row 1: Length from Proximal Joint to CoM (L_joint_to_CoM)
    - Row 2: Length from CoM to Distal Joint (L_CoM_to_joint)

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the average CoM sub-lengths.
    - proximal_segment_col (str): Column name of the proximal segment (e.g., 'Left Thigh').
    - distal_segment_col (str): Column name of the distal segment (e.g., 'Left Shank').

    Returns:
    - np.array: [L_CoM_to_joint_proximal, 0, -L_joint_to_CoM_distal, 0]
    """

    # L_CoM_to_joint_proximal: Length of the proximal segment from its CoM to the joint (Distal end).
    # This is the value from Row 2 of the proximal segment's column.
    L_CoM_to_joint_proximal = df.loc[2, proximal_segment_col]

    # L_joint_to_CoM_distal: Length of the distal segment from the joint (Proximal end) to its CoM.
    # This is the value from Row 1 of the distal segment's column.
    L_joint_to_CoM_distal = df.loc[1, distal_segment_col]

    # Create the array in the required format [L1, 0, -L2, 0]
    result_array = np.array([
        L_CoM_to_joint_proximal,
        0.0,
        -L_joint_to_CoM_distal,
        0.0
    ])

    return result_array

### Creating the revolut joint relations out of body distances to connected joint


In [51]:
# Creating a simple dataframe that stores the mubokap information for the revolut joint configuration
# The dataframe created is exported into a .tsv file that needs to be extended with the body IDs from the rigid body definition in MuboKap
joint_columns = ['Neck', 'Left Shoulder', 'Left Elbow', 'Right Shoulder', 'Right Elbow', 'Right Hip', 'Right Knee', 'Right Ankle', 'Right Toe Joint', 'Left Hip', 'Left Knee', 'Left Ankle', 'Left Toe Joint']

rev_joints = pd.DataFrame(columns=joint_columns)


# Elbow Joints (Upperarm -> Forearm)
rev_joints['Right Elbow'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Right Upper Arm',
    distal_segment_col='Right Forearm'
)

rev_joints['Left Elbow'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Left Upper Arm',
    distal_segment_col='Left Forearm'
)

# Hip Joints (Trunk -> Thigh)
rev_joints['Left Hip'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Trunk',
    distal_segment_col='Left Thigh'
)

rev_joints['Right Hip'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Trunk',
    distal_segment_col='Right Thigh'
)

# Knee Joints (Thigh -> Shank)
rev_joints['Left Knee'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Left Thigh',
    distal_segment_col='Left Shank'
)

rev_joints['Right Knee'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Right Thigh',
    distal_segment_col='Right Shank'
)

# Ankle Joints (Shank -> Foot)
rev_joints['Left Ankle'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Left Shank',
    distal_segment_col='Left Foot'
)

rev_joints['Right Ankle'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Right Shank',
    distal_segment_col='Right Foot'
)

# Toe Joints (Foot -> Toe)
rev_joints['Left Toe Joint'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Left Foot',
    distal_segment_col='Left Toe'
)

rev_joints['Right Toe Joint'] = create_joint_length_array(
    avg_lengths,
    proximal_segment_col='Right Foot',
    distal_segment_col='Right Toe'
)

# Neck Joints (Trunk -> Neck)
rev_joints['Neck'] = np.array([
        -avg_lengths.loc[1, 'Trunk'],
        0.0,
        -avg_lengths.loc[1, 'Neck'],
        0.0
    ])

# Shoulder Joints (Trunk -> Upper Arm)
rev_joints['Left Shoulder'] = np.array([
        -avg_lengths.loc[1, 'Trunk'],
        0.0,
        -avg_lengths.loc[1, 'Left Upper Arm'],
        0.0
    ])

rev_joints['Right Shoulder'] = np.array([
        -avg_lengths.loc[1, 'Trunk'],
        0.0,
        -avg_lengths.loc[1, 'Right Upper Arm'],
        0.0
    ])

print("\nResult Array for Joint Analysis:")
print(rev_joints)


Result Array for Joint Analysis:
       Neck  Left Shoulder  Left Elbow  Right Shoulder  Right Elbow  \
0 -0.278162      -0.278162    0.204588       -0.278162     0.206726   
1  0.000000       0.000000    0.000000        0.000000     0.000000   
2 -0.162556      -0.158156   -0.110444       -0.159809    -0.106800   
3  0.000000       0.000000    0.000000        0.000000     0.000000   

   Right Hip  Right Knee  Right Ankle  Right Toe Joint  Left Hip  Left Knee  \
0   0.278162    0.238101     0.253489         0.060850  0.278162   0.238495   
1   0.000000    0.000000     0.000000         0.000000  0.000000   0.000000   
2  -0.181830   -0.193581    -0.060850        -0.049736 -0.182131  -0.194579   
3   0.000000    0.000000     0.000000         0.000000  0.000000   0.000000   

   Left Ankle  Left Toe Joint  
0    0.254795        0.060663  
1    0.000000        0.000000  
2   -0.060663       -0.047922  
3    0.000000        0.000000  


## Filter Dynamic Data

In this section of the code, dynamic trial data is:
    - read
    - projected onto sagittal plane
    - converted from mm to m
    - assigned to the same df format as the static data (P1-P15)
    - filtered using a 2nd order butterworth filter to smoothen the signal for further processing


In [52]:
# assessing the .tsv file that holds the kinematic data ({x,y,z} coordinates for each marker)
dynamic_3D = pd.read_csv('Trial0003_str11.tsv', delimiter='\t', skiprows=10)

dynamic_3D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Frame             149 non-null    int64  
 1   Time              149 non-null    float64
 2   1 - Head X        149 non-null    float64
 3   1 - Head Y        149 non-null    float64
 4   1 - Head Z        149 non-null    float64
 5   2 - L_Shoulder X  149 non-null    float64
 6   2 - L_Shoulder Y  149 non-null    float64
 7   2 - L_Shoulder Z  149 non-null    float64
 8   3 - L_Elbow X     149 non-null    float64
 9   3 - L_Elbow Y     149 non-null    float64
 10  3 - L_Elbow Z     149 non-null    float64
 11  4 - L_Wrist X     149 non-null    float64
 12  4 - L_Wrist Y     149 non-null    float64
 13  4 - L_Wrist Z     149 non-null    float64
 14  5 - R_Shoulder X  149 non-null    float64
 15  5 - R_Shoulder Y  149 non-null    float64
 16  5 - R_Shoulder Z  149 non-null    float64
 1

In [53]:
# Project 3D data onto 2D plane (dropping y-axis information)

# The `axis=1` tells pandas to drop columns, not rows.
dynamic_2D = dynamic_3D.drop(columns=columns_to_drop, axis=1)

selected_columns = dynamic_2D.columns[2:]

for col in selected_columns:
    dynamic_2D[col] = dynamic_2D[col] / 1000

# 3. Create the new DataFrame with coordinate tuples
dynamic_df = create_coordinate_columns(dynamic_2D)

print("\nDataFrame with only Coordinate Tuples:\n", dynamic_df)



DataFrame with only Coordinate Tuples:
                                 1 - Head_Coord  \
0               (0.597858, 1.7594020000000001)   
1               (0.5884109999999999, 1.757504)   
2                         (0.578699, 1.755065)   
3                          (0.568923, 1.75316)   
4                         (0.559077, 1.751549)   
..                                         ...   
144  (-0.9008189999999999, 1.7452180000000002)   
145  (-0.9141330000000001, 1.7473299999999998)   
146                      (-0.926914, 1.749233)   
147            (-0.939448, 1.7510599999999998)   
148                      (-0.952332, 1.753354)   

                   10 - L_Ankle_Coord                 11 - L_Heel_Coord  \
0                (0.933793, 0.064916)    (0.9925259999999999, 0.026298)   
1      (0.9334450000000001, 0.065523)              (0.992423, 0.027834)   
2      (0.9330900000000001, 0.066451)                (0.992325, 0.0295)   
3                (0.932986, 0.067496)              (0.9921

In [54]:
# Composing Dataframe 'Points' for biomechanical model
num_rows_dynamic = len(dynamic_2D)

# Create an array of NaNs with the correct shape (Rows x Columns)
# The data type is set to float (dtype='float64')
data = np.full((num_rows_dynamic, len(column_names_points)), np.nan)

# 3. Create the final DataFrame
dynamic_points = pd.DataFrame(
    data,
    columns=column_names_points
)

# --- Display the result ---
print("\n--- New Target DataFrame Layout ---")
print(f"Shape: {dynamic_points.shape} (Rows x Columns)")
print(dynamic_points.head()) # Shows the first few rows

# Determining and storing shoulder midpoint data to points dataframe
x_com_shoulder, z_com_shoulder = calculate_segment_com(
    dynamic_df,
    proximal_col='2 - L_Shoulder_Coord',
    distal_col='5 - R_Shoulder_Coord',
    com_ratio=0.5
)
dynamic_points['P2'] = package_coordinates(x_com_shoulder, z_com_shoulder)

x_com_hip, z_com_hip = calculate_segment_com(
    dynamic_df,
    proximal_col='8 - L_Hip_Coord',
    distal_col='14 - R_Hip_Coord',
    com_ratio=0.5
)
dynamic_points['P7'] = package_coordinates(x_com_hip, z_com_hip)


--- New Target DataFrame Layout ---
Shape: (149, 15) (Rows x Columns)
   P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  P11  P12  P13  P14  P15
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN


In [55]:
dynamic_points['P1'] = dynamic_df['1 - Head_Coord']
dynamic_points['P3'] = dynamic_df['3 - L_Elbow_Coord']
dynamic_points['P4'] = dynamic_df['4 - L_Wrist_Coord']
dynamic_points['P5'] = dynamic_df['6 - R_Elbow_Coord']
dynamic_points['P6'] = dynamic_df['7 - R_Wrist_Coord']
dynamic_points['P8'] = dynamic_df['15 - R_Knee_Coord']
dynamic_points['P9'] = dynamic_df['16 - R_Ankle_Coord']
dynamic_points['P10'] = dynamic_df['18 - R_Meta_V_Coord']
dynamic_points['P11'] = dynamic_df['19 - R_Toe_II_Coord']
dynamic_points['P12'] = dynamic_df['9 - L_Knee_Coord']
dynamic_points['P13'] = dynamic_df['10 - L_Ankle_Coord']
dynamic_points['P14'] = dynamic_df['12 - L_Meta_V_Coord']
dynamic_points['P15'] = dynamic_df['13 - L_Toe_II_Coord']

print(dynamic_points)

                                            P1  \
0               (0.597858, 1.7594020000000001)   
1               (0.5884109999999999, 1.757504)   
2                         (0.578699, 1.755065)   
3                          (0.568923, 1.75316)   
4                         (0.559077, 1.751549)   
..                                         ...   
144  (-0.9008189999999999, 1.7452180000000002)   
145  (-0.9141330000000001, 1.7473299999999998)   
146                      (-0.926914, 1.749233)   
147            (-0.939448, 1.7510599999999998)   
148                      (-0.952332, 1.753354)   

                                            P2  \
0                        (0.720383, 1.4609275)   
1               (0.7104459999999999, 1.458421)   
2              (0.7002740000000001, 1.4557995)   
3                         (0.690236, 1.453239)   
4                         (0.680026, 1.450949)   
..                                         ...   
144           (-0.7879765000000001, 1.4410915)   

## Filtering of the point data

In [56]:
dummy_df = pd.DataFrame()

dynamic_filtered = pd.DataFrame(columns=column_names_points)

for column in column_names_points:

    dummy_df[['X', 'Z']] = dynamic_points[column].apply(pd.Series)

    x_filtered = signal.filtfilt(b, a, dummy_df['X'])
    z_filtered = signal.filtfilt(b, a, dummy_df['Z'])

    dynamic_filtered[column] = package_coordinates(x_filtered, z_filtered)

print(dynamic_filtered)

                                            P1  \
0      (0.5991300719591384, 1.759461561883871)   
1      (0.5890652968678831, 1.757342713466981)   
2     (0.5790197027199653, 1.7552433214606813)   
3     (0.5689787188371909, 1.7531945856293105)   
4     (0.5589199891635729, 1.7512273071629754)   
..                                         ...   
144  (-0.9025884146470889, 1.7459780146456139)   
145  (-0.9157322323931724, 1.7475667570133546)   
146  (-0.9290022830492671, 1.7493676983551831)   
147  (-0.9422718974255796, 1.7513022560042593)   
148   (-0.9553680229769203, 1.753287321762064)   

                                            P2  \
0     (0.7216836728577354, 1.4610169772003092)   
1     (0.7112554521010193, 1.4584525629801508)   
2     (0.7008576590800868, 1.4559204882514027)   
3     (0.6904868430606024, 1.4534627183618325)   
4     (0.6801274432359119, 1.4511185622043727)   
..                                         ...   
144  (-0.7893702705253651, 1.4424320543320885)   

In [58]:
def calculate_segment_orientation(proximal_point, distal_point):
    """
    Calculates the orientation angle (theta) of a segment (vector from proximal
    to distal point) with respect to the positive X-axis of the global reference frame.

    The angle is returned in radians.

    Parameters:
    - proximal_point (tuple or list): (X_p, Z_p) coordinates of the proximal end.
    - distal_point (tuple or list): (X_d, Z_d) coordinates of the distal end.

    Returns:
    - float: Orientation angle theta in radians.
    """

    # 1. Calculate the components of the segment vector
    # Vector V = Distal - Proximal

    # X component (horizontal change)
    delta_x = distal_point[0] - proximal_point[0]

    # Z component (vertical change - this is often labeled Y in typical math,
    # but since you specified Z as the second coordinate, we use Z here)
    delta_z = distal_point[1] - proximal_point[1]

    # 2. Use the atan2 function
    # atan2(y, x) gives the angle in radians, correctly handling quadrants.
    # In the X-Z plane, Z is the vertical component (like Y), and X is the horizontal.
    theta_radians = np.arctan2(delta_z, delta_x)

    # 3. Convert angle from (-pi, pi] range to [0, 2*pi) range
    # Check for negative angle and add 2*pi (360 degrees) to make it positive
    if theta_radians < 0:
        theta_radians += 2 * np.pi

    # 3. Convert to degrees
    # theta_degrees = np.degrees(theta_radians)

    return theta_radians # theta_degrees

# Defining 14 sequential segments:
segment_config = {
    'Neck': ('P2', 'P1'),
    'Trunk': ('P2', 'P7'),
    'Right Upper Arm': ('P2', 'P5'),
    'Right Forearm': ('P5', 'P6'),
    'Left Upper Arm': ('P2', 'P3'),
    'Left Forearm': ('P3', 'P4'),
    'Right Thigh': ('P7', 'P8'),
    'Right Shank': ('P8', 'P9'),
    'Right Foot': ('P9', 'P10'),
    'Right Toe': ('P10', 'P11'),
    'Left Thigh': ('P7', 'P12'),
    'Left Shank': ('P12', 'P13'),
    'Left Foot': ('P13', 'P14'),
    'Left Toe': ('P14', 'P15'),
}

dynamic_bodies_config = []
# Iterate over the segment configuration dictionary
for segment_name, (proximal_key, distal_key) in segment_config.items():

    # Extract the (X, Z) coordinates from the main point DataFrame
    # .loc[key].values returns the coordinates as a NumPy array [X, Z]
    proximal_point = dynamic_points.loc[0, proximal_key]
    distal_point = dynamic_points.loc[0, distal_key]

    # Calculate the orientation angle using the function
    orientation_angle = calculate_segment_orientation(proximal_point, distal_point)

    # Store the results in the list
    dynamic_bodies_config.append({
        'Segment Name': segment_name,
        'Orientation Angle': orientation_angle,
        'Orientation Angle (deg)': np.degrees(orientation_angle)
    })

# Create the final DataFrame with 14 rows
df_initial_orientation = pd.DataFrame(dynamic_bodies_config)

print(df_initial_orientation)

       Segment Name  Orientation Angle  Orientation Angle (deg)
0              Neck           1.960325               112.318350
1             Trunk           4.770879               273.351238
2   Right Upper Arm           5.015804               287.384387
3     Right Forearm           4.294482               246.055709
4    Left Upper Arm           4.607854               264.010609
5      Left Forearm           4.054397               232.299808
6       Right Thigh           4.281914               245.335603
7       Right Shank           4.450845               255.014624
8        Right Foot           3.184705               182.470149
9         Right Toe           2.805262               160.729690
10       Left Thigh           4.872252               279.159470
11       Left Shank           4.970038               284.762177
12        Left Foot           3.483469               199.588098
13         Left Toe           3.203104               183.524325
